# Оптимизация функции с использованием генетического алгоритма

В этом Jupyter Notebook мы будем использовать библиотеку DEAP для оптимизации функции $$ y = x^2 + 4 $$

## Установка DEAP

Для начала установим необходимую библиотеку DEAP, если она еще не установлена.

In [2]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.9 MB/s eta 0:00:00


## Импорт необходимых библиотек


In [ ]:
from deap import base, algorithms
from deap import creator
from deap import tools
import random
import numpy as np

## Параметры генетического алгоритма

Зададим основные параметры нашего генетического алгоритма.


In [ ]:
LOW, UP = -5, 5
ETA = 20
LENGTH_CHROM = 1 # длинна хромосомы (x)
POPULATION_SIZE = 200     # количество индивидуумов в популяции
P_CROSSOVER = 0.9         # вероятность скрещивания
P_MUTATION = 0.2          # вероятность мутации индивидуума
MAX_GENERATION = 50       # максимальное количество поколения
HALL_OF_FAME_SIZE = 5

## Создание классов и функций

In [ ]:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def randomPoint(a, b):
    return [random.uniform(a, b)]

toolbox = base.Toolbox()
toolbox.register("randomPoint", randomPoint, LOW, UP)
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomPoint)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

population = toolbox.populationCreator(n=POPULATION_SIZE)

## Определение функции приспособленности

In [ ]:
def minFunction(individual):
    x = individual
    f = np.power(x, 2) + 4
    return f,

toolbox.register("evaluate", minFunction)

## Регистрация операторов генетического алгоритма

In [ ]:
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=LOW, up=UP, eta=ETA)
toolbox.register("mutate", tools.mutPolynomialBounded, low=LOW, up=UP, eta=ETA, indpb=1.0/LENGTH_CHROM)

## Запуск генетического алгоритма

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min)
stats.register("avg", np.mean)

population, logbook = algorithms.eaSimple(population, toolbox,
                                          cxpb=P_CROSSOVER,
                                          mutpb=P_MUTATION,
                                          ngen=MAX_GENERATION,
                                          halloffame=hof,
                                          stats=stats,
                                          verbose=True)

## Вывод результатов

In [ ]:
minFitnessValues, meanFitnessValues = logbook.select("min", "avg")

best = hof.items[0]
print("Лучший индивидуум: ", best)
print("Значение функции в лучшей точке: ", minFunction(best))